In [10]:
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 74aebb552ea7... 100% ▕████████████████▏  68 MB                         
pulling ef1438627c47... 100% ▕████████████████▏  190 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6f85a640a97c... 100% ▕████████████████▏ 807 MB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 6c0b08d96525... 100% ▕████████████████▏   65 B                         
pulling 60f68b1aefd0... 100% ▕████████████████▏  193 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sezer\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [12]:
dataset = []
with open('feedback_data.json', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

Loaded 34 entries


In [13]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

In [14]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')


Added chunk 1/34 to the database
Added chunk 2/34 to the database
Added chunk 3/34 to the database
Added chunk 4/34 to the database
Added chunk 5/34 to the database
Added chunk 6/34 to the database
Added chunk 7/34 to the database
Added chunk 8/34 to the database
Added chunk 9/34 to the database
Added chunk 10/34 to the database
Added chunk 11/34 to the database
Added chunk 12/34 to the database
Added chunk 13/34 to the database
Added chunk 14/34 to the database
Added chunk 15/34 to the database
Added chunk 16/34 to the database
Added chunk 17/34 to the database
Added chunk 18/34 to the database
Added chunk 19/34 to the database
Added chunk 20/34 to the database
Added chunk 21/34 to the database
Added chunk 22/34 to the database
Added chunk 23/34 to the database
Added chunk 24/34 to the database
Added chunk 25/34 to the database
Added chunk 26/34 to the database
Added chunk 27/34 to the database
Added chunk 28/34 to the database
Added chunk 29/34 to the database
Added chunk 30/34 to th

In [15]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [16]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [17]:
import concurrent.futures
import time

# Mock retrieve function for testing purposes
def retrieve(query):
    # Simulate a delay for testing
    time.sleep(2)
    return [("Sample chunk 1", 0.9), ("Sample chunk 2", 0.8)]

def retrieve_with_timeout(query, timeout=10):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(retrieve, query)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print(f"Retrieve function timed out after {timeout} seconds")
            return []

# Use a simple predefined input for testing
input_query = "Give me summary how is the logistics at the moment?"

start_time = time.time()
retrieved_knowledge = retrieve_with_timeout(input_query, timeout=10)  # Set your desired timeout here
end_time = time.time()

print(f"Retrieve function took {end_time - start_time:.2f} seconds")

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
    print(f' - (similarity: {similarity:.2f}) {chunk}')

formatted_chunks = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])
result = f"{formatted_chunks}"

instruction_prompt = f"""You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{formatted_chunks}"""

print(instruction_prompt)

Retrieve function took 2.02 seconds
Retrieved knowledge:
 - (similarity: 0.90) Sample chunk 1
 - (similarity: 0.80) Sample chunk 2
You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - Sample chunk 1
 - Sample chunk 2


In [18]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)


print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
I'm happy to provide you with an update on the logistics situation.

Based on my analysis of Sample Chunk 1 and Sample Chunk 2, it appears that some shipping carriers are experiencing disruptions due to various factors such as increased demand, staffing shortages, and supply chain issues. The exact impact is still unfolding, but overall, there are reports of delays and inconvenience for customers who rely on these services.

For example, Sample Chunk 1 mentioned that some carriers were struggling with high volumes of packages, leading to congestion in their sorting facilities. Similarly, Sample Chunk 2 noted that transportation costs were rising due to inflationary pressures.

It's worth noting that the situation is not uniform across all regions or types of carriers, and some may be experiencing more disruptions than others.

Overall, it seems that logistics are facing a challenging time as carriers navigate these challenges, but it's too early to say when everything